<a href="https://colab.research.google.com/github/joestarcsz/My_IA_chat/blob/main/Arquivos_com_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!pip install -U -q google-generativeai

In [56]:
#importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

GOOGLE_API_KEY=(sua api key)
genai.configure(api_key=GOOGLE_API_KEY)

In [57]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [58]:
#Exemplo de embedding
title = "jojos bizarre adventure"
sample_text = ("Titulo: JoJo`s Bizarre Adventure"
"\n"
"Artigo completo:\n"
"\n"
"JoJo`s Bizarre Adventure: O manga bizarramente lindo")

embeddings = genai.embed_content(model="models/text-embedding-004",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [-0.032406654, 0.019228984, -0.017817484, -0.013887949, 0.0009828879, 0.02929227, -0.01844947, -0.007577441, 0.06164216, 0.048352428, 0.0002342364, -0.027805448, 0.07294481, 0.04233555, 0.005496788, -0.06323784, -0.0347936, 0.03576358, -0.113357246, 0.06386702, 0.084943265, -0.011665757, 0.054751437, 0.0019417704, -0.053929735, 0.04928021, -0.01070433, 0.027037209, -0.023033382, -0.007855034, -0.005637675, 0.027178917, 0.024300301, -0.011422689, 0.022055447, -0.0015269909, -0.040008243, 0.041706063, 0.048413333, -0.08064162, -0.064276636, 0.018062215, -0.039842635, -0.0157451, -0.026646994, 0.0021428312, 0.049412116, 0.056745157, 0.0028433756, 0.049243703, 0.02712736, 0.056288507, -0.12084838, 0.011589132, -0.0335742, 0.029205354, -0.02600994, -0.008779029, -0.002734928, 0.00031001124, 0.013716026, -0.011060044, -0.020851571, 0.009610175, -0.0034534938, -0.044348154, -0.052442584, 0.0062425733, -0.027482707, -0.0035567032, -0.013362967, -0.009288768, -0.05051108, 0.012836

In [59]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "title": "JoJo`s Bizarre Adventure",
    "content": "JoJo`s Bizarre Adventure: O manga bizarramente lindo" }

DOCUMENT2 = {
    "title": "JoJo`s Bizarre Adventure",
    "content": "JoJo`s Bizarre Adventure: O anime que utiliza nome de bandas e musicas de rock para seus poderes" }

DOCUMENT3 = {
    "title": "JoJo`s Bizarre Adventure",
    "content": "JoJo`s Bizarre Adventure: A obra de araki" }

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [72]:
df = pd.DataFrame(documents)
df.columns = ['title', 'conteudo']
df

,title,conteudo
0,JoJo`s Bizarre Adventure,JoJo`s Bizarre Adventure: O manga bizarramente...
1,JoJo`s Bizarre Adventure,JoJo`s Bizarre Adventure: O anime que utiliza ...
2,JoJo`s Bizarre Adventure,JoJo`s Bizarre Adventure: A obra de araki


In [73]:
model = "models/text-embedding-004"

In [74]:
def embed_fn(title, text):
  embeddings = genai.embed_content(model=model,
                                   content=text,
                                   title=title,
                                   task_type="RETRIEVAL_DOCUMENT")["embedding"]
  return embeddings

In [75]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["title"], row["conteudo"]), axis=1)
df

,title,conteudo,Embeddings
0,JoJo`s Bizarre Adventure,JoJo`s Bizarre Adventure: O manga bizarramente...,"[-0.04352767, 0.014090506, -0.015823036, -0.02..."
1,JoJo`s Bizarre Adventure,JoJo`s Bizarre Adventure: O anime que utiliza ...,"[-0.06720934, 0.0137965325, -0.045251098, -0.0..."
2,JoJo`s Bizarre Adventure,JoJo`s Bizarre Adventure: A obra de araki,"[-0.035082765, 0.014947801, -0.02203251, -0.00..."


In [76]:
def gerar_e_buscar_consulta(consulta, base , model):
  consulta_embedding = genai.embed_content(model=model,
                                   content=consulta,
                                   task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), consulta_embedding)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["conteudo"]

In [82]:
consulta = "Oque Jojo utiliza para seus poderes?"

trecho = gerar_e_buscar_consulta(consulta, df , model)
print(trecho)

JoJo`s Bizarre Adventure: O anime que utiliza nome de bandas e musicas de rock para seus poderes


In [83]:
generation_config = {
    "temperature": 0.5,
    "candidate_count": 1,
}

In [85]:
prompt = f"Reenscreva esse texto de uma forma mais descontraida, sem adicionar informações que não fazem parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

JoJo's Bizarre Adventure: O anime onde os poderes são uma playlist de rock!
